In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
spark


SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
# a) Create Airline Pysaprk sql dataFrame.  
from pyspark.sql.functions import *
flight_delay_df = spark.read.csv('dbfs:/FileStore/tables/Flights_Delay.csv', header=True, inferSchema=True)
flight_delay_df.createOrReplaceTempView("flightdelaytable")
flight_delay_df.count()
flight_delay_df.select([count(when(isnull(col), col)).alias(col) for col in flight_delay_df.columns]).show()

+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
| ID|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+-----------

In [0]:
#b) Describe the table schema & show 10 rows of Dataset
flight_delay_df.printSchema()
flight_delay_df.show(10)

root
 |-- ID: integer (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true

In [0]:
# c) Average arrival delay caused by airlines.
spark.sql("select AIRLINE, mean(ARRIVAL_DELAY) from flightdelaytable group by AIRLINE").show()

+-------+-------------------+
|AIRLINE|mean(ARRIVAL_DELAY)|
+-------+-------------------+
|     UA|  6.697221614526362|
|     NK| 14.206426484907498|
|     AA|  8.386631979187513|
|     EV| 10.884270870655678|
|     B6|  13.95852534562212|
|     DL| 2.8144726712856043|
|     OO| 10.154792043399638|
|     F9| 24.103448275862068|
|     US|  5.977315185481719|
|     MQ| 19.231592604605904|
|     HA|  4.072423398328691|
|     AS| -1.531766200762389|
|     VX|  5.128571428571429|
|     WN|  3.697840458351697|
+-------+-------------------+



In [0]:
# d) Days of months with average Arrival delays.
spark.sql("select DAY, mean(ARRIVAL_DELAY) from flightdelaytable group by DAY order by DAY").show()

+---+--------------------+
|DAY| mean(ARRIVAL_DELAY)|
+---+--------------------+
|  1|  14.807807807807809|
|  2|  15.046014790468364|
|  3|  18.141541038525965|
|  4|  17.157790927021697|
|  5|   16.23861262014208|
|  6|  10.608832807570979|
|  7|  2.8309417040358746|
|  8|   5.232349165596919|
|  9|   4.421887390959556|
| 10|-0.04705882352941...|
| 11|  3.9912935323383083|
| 12|   11.24892703862661|
| 13|  3.3769751693002257|
| 14|  1.3299319727891157|
| 15|   2.966753585397653|
| 16|   9.124321062160531|
| 17|   8.761435608726249|
| 18|  3.5693430656934306|
| 19|  1.6344282238442822|
| 20|  3.8770149253731345|
+---+--------------------+
only showing top 20 rows



In [0]:
# e) Days of months with median Departure delays.
spark.sql("select DAY,percentile_approx(DEPARTURE_DELAY,0.5) as DepartureDelayMEDIAN from flightdelaytable group by DAY order by DAY ASC").show()

+---+--------------------+
|DAY|DepartureDelayMEDIAN|
+---+--------------------+
|  1|                   0|
|  2|                   2|
|  3|                   1|
|  4|                   0|
|  5|                   1|
|  6|                   0|
|  7|                  -2|
|  8|                  -1|
|  9|                  -1|
| 10|                  -2|
| 11|                  -2|
| 12|                   0|
| 13|                  -1|
| 14|                  -2|
| 15|                  -1|
| 16|                  -1|
| 17|                   0|
| 18|                  -2|
| 19|                  -2|
| 20|                  -2|
+---+--------------------+
only showing top 20 rows



In [0]:
# f) Arrange weekdays with respect to the average arrival delays.
spark.sql("select DAY_OF_WEEK, mean(ARRIVAL_DELAY) as Avg_Arrival_delay from flightdelaytable group by DAY_OF_WEEK order by Avg_Arrival_delay").show()

+-----------+------------------+
|DAY_OF_WEEK| Avg_Arrival_delay|
+-----------+------------------+
|          6| 4.888689138576779|
|          3| 5.587079407806191|
|          5| 6.010538373424971|
|          4| 7.174969021065675|
|          2| 8.033644102148358|
|          7|10.110840438489646|
|          1|10.807447207297264|
+-----------+------------------+



In [0]:
# g) Show Analysis for each month with total number of cancellations. (Descending).
spark.sql("select MONTH, sum(CANCELLED) TotalFlightsCancelled from flightdelaytable group by MONTH order by TotalFlightsCancelled DESC").show()

+-----+---------------------+
|MONTH|TotalFlightsCancelled|
+-----+---------------------+
|    2|                 1087|
|    1|                  608|
|    3|                  456|
+-----+---------------------+



In [0]:
# h) Show the top 10 busiest airport. [Hint: Sum Total of Arrival and Departures from an Airport].
spark.sql("select ORIGIN_AIRPORT as Aiports, count(ORIGIN_AIRPORT)+count(DESTINATION_AIRPORT) as cnt from flightdelaytable group by ORIGIN_AIRPORT order by cnt DESC").show(10)

+-------+----+
|Aiports| cnt|
+-------+----+
|    ATL|7154|
|    ORD|5792|
|    DFW|5602|
|    LAX|4094|
|    DEN|4070|
|    IAH|3224|
|    PHX|3178|
|    SFO|3010|
|    LAS|2832|
|    MCO|2400|
+-------+----+
only showing top 10 rows



In [0]:
# i) Show the airlines that make the maximum number of cancellations.
spark.sql("select AIRLINE,sum(CANCELLED) as No_Of_Cancellations from flightdelaytable group by AIRLINE order by No_Of_Cancellations DESC").show(5)

+-------+-------------------+
|AIRLINE|No_Of_Cancellations|
+-------+-------------------+
|     MQ|                414|
|     WN|                358|
|     EV|                312|
|     AA|                241|
|     DL|                177|
+-------+-------------------+
only showing top 5 rows



In [0]:
# j) Find and order airlines in descending that make the most number of diversions.
spark.sql("select AIRLINE,sum(DIVERTED) as No_Of_Diversions from flightdelaytable group by AIRLINE order by No_Of_Diversions DESC").show(5)

+-------+----------------+
|AIRLINE|No_Of_Diversions|
+-------+----------------+
|     WN|              35|
|     OO|              25|
|     EV|              22|
|     DL|              18|
|     B6|              16|
+-------+----------------+
only showing top 5 rows



In [0]:
# k) Show days of month that seen the most number of diversion.
spark.sql("select DAY, sum(DIVERTED) as No_Of_Diversions from flightdelaytable group by DAY order by No_Of_Diversions DESC").show()

+---+----------------+
|DAY|No_Of_Diversions|
+---+----------------+
|  2|              15|
|  1|              13|
|  4|              12|
|  5|              11|
|  9|               9|
| 14|               8|
|  6|               7|
|  7|               6|
| 23|               6|
|  3|               5|
| 30|               5|
| 11|               5|
|  8|               5|
| 18|               5|
| 16|               4|
| 28|               4|
| 20|               4|
| 12|               4|
| 21|               4|
| 31|               3|
+---+----------------+
only showing top 20 rows



In [0]:
# l) Calculate mean and standard deviation of departure delay for all flights in minutes.
spark.sql("select AIRLINE, mean(DEPARTURE_DELAY), stddev(DEPARTURE_DELAY) from flightdelaytable group by AIRLINE").show()

+-------+---------------------+-----------------------+
|AIRLINE|mean(DEPARTURE_DELAY)|stddev(DEPARTURE_DELAY)|
+-------+---------------------+-----------------------+
|     UA|   14.293129770992367|      36.36679305829193|
|     NK|   15.580739299610896|     46.004490473829854|
|     AA|   11.502293120638086|      50.58770621277648|
|     EV|   11.534709861160556|       40.6022135127612|
|     B6|   16.068664169787766|     44.452210632841826|
|     DL|    9.938812083973374|      44.57836834851712|
|     OO|   11.601510519690702|     41.898251511636026|
|     F9|   23.514687100893997|      55.21644901488807|
|     US|    7.806271591815041|     29.947392168792227|
|     MQ|    17.07297384565709|      43.46879359686762|
|     HA|   1.1835883171070931|      30.28223244986787|
|     AS|   2.3365079365079366|      29.14656093746487|
|     VX|    9.857142857142858|     35.175272138214424|
|     WN|   10.115161630358397|      28.66211787470035|
+-------+---------------------+-----------------

In [0]:
# m) Calculate mean and standard deviation of arrival delay for all flights in minutes.
spark.sql("select AIRLINE, mean(ARRIVAL_DELAY), stddev(ARRIVAL_DELAY) from flightdelaytable group by AIRLINE").show()

+-------+-------------------+---------------------+
|AIRLINE|mean(ARRIVAL_DELAY)|stddev(ARRIVAL_DELAY)|
+-------+-------------------+---------------------+
|     UA|  6.697221614526362|    38.96583521422204|
|     NK| 14.206426484907498|    47.57688184145277|
|     AA|  8.386631979187513|    53.56770860254512|
|     EV| 10.884270870655678|    43.39199993434066|
|     B6|  13.95852534562212|    47.64386997495008|
|     DL| 2.8144726712856043|    46.95730281739299|
|     OO| 10.154792043399638|    43.76036682697078|
|     F9| 24.103448275862068|    56.26804479748733|
|     US|  5.977315185481719|    34.10900474893441|
|     MQ| 19.231592604605904|    46.39998574481305|
|     HA|  4.072423398328691|    32.38330048967069|
|     AS| -1.531766200762389|    31.37282637924461|
|     VX|  5.128571428571429|    40.86556219200457|
|     WN|  3.697840458351697|   31.225094725977485|
+-------+-------------------+---------------------+



In [0]:
# n) Find all diverted Route from a source to destination Airport & which route is the most diverted.
spark.sql("select ORIGIN_AIRPORT,DESTINATION_AIRPORT, sum(DIVERTED) as No_Of_Diversions from flightdelaytable group by ORIGIN_AIRPORT,DESTINATION_AIRPORT order by No_Of_Diversions DESC").show()

+--------------+-------------------+----------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|No_Of_Diversions|
+--------------+-------------------+----------------+
|           IAH|                ASE|               2|
|           STT|                PHL|               2|
|           PHL|                SAN|               2|
|           HOU|                DAL|               2|
|           ORD|                ASE|               2|
|           JFK|                EGE|               2|
|           CLT|                IAH|               2|
|           JFK|                SEA|               2|
|           TPA|                LGA|               2|
|           MCO|                PVD|               1|
|           ATL|                ASE|               1|
|           FLL|                PVD|               1|
|           ATL|                LGA|               1|
|           IAH|                ISN|               1|
|           KOA|                SFO|               1|
|           SLC|            

In [0]:
# o) Finding AIRLINES with its total flight count, total number of flights arrival delayed by more than 30 Minutes, % of such flights delayed by more than 30 minutes when it is not Weekends with minimum count of flights from Airlines by more than 10. Also Exclude some of Airlines 'AK', 'HI', 'PR', 'VI' and arrange output in descending order by % of such count of flights.
spark.sql("select  count(AIRLINE) as FlightCount from flightdelaytable group by AIRLINE").show()
#Not finished.

+-----------+
|FlightCount|
+-----------+
|       4701|
|       1048|
|       5250|
|       5916|
|       2548|
|       7989|
|       5708|
|        794|
|       3925|
|       3502|
|        722|
|       1586|
|        573|
|      11738|
+-----------+

